In [1]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

# getting articles text
- to merge abstarct and title one needs to concat them with new line
- 3 possible cases of abstarcts (1. Abstarct and text of it, 2. Abstract divided into section 3. Diffrent division for the one document (14611657))
- Maxline needed to get only relevant annoatations


In [2]:
articles_folder = os.path.join('articles','txt')

The first line is a title. Then we wait for the abstract. The text of the abstarct is everything until the empty line occure

In [20]:
texts = pd.DataFrame(columns = ['PMID','Type','Content','MaxLine'])
s = []
for file in os.listdir(articles_folder):
    if file[-3:] == 'txt':
        name = file[:-4]
        with open(os.path.join(articles_folder,file),'r',encoding='utf-8') as f:
            article  = pd.DataFrame()
            lines = list(f.readlines())
            texts.loc[len(texts)] = {'PMID':name, 'Type':'t','Content':lines[0]}
            if lines[4] not in [ 'Background\n','Backgroud\n','Introduction\n'] and name != '14611657':
                texts.loc[len(texts)] = {'PMID':name, 'Type':'a','Content':''.join(lines[1:5]), 'MaxLine':len(''.join(lines[:5]))}
            else:
                if name == '14611657':
                    txt = ''.join(lines[1:7])
                    for i,line in enumerate(lines[7:]):
                        if line == 'Background\n' or line == 'Introduction\n':
                            break
                        txt = ''.join([txt,line])
                    texts.loc[len(texts)] = {'PMID':name, 'Type':'a','Content':txt,'MaxLine':len(''.join(lines[:(7+i)]))}
                else:
                    txt = ''.join(lines[1:5])
                    for i,line in enumerate(lines[5:]):
                        if line == 'Background\n' or line == 'Introduction\n':
                            break
                        txt = ''.join([txt,line])
                    texts.loc[len(texts)] = {'PMID':name, 'Type':'a','Content':txt,'MaxLine':len(''.join(lines[:(5+i)]))}
                        

        


In [5]:
texts.to_csv(os.path.join('data', 'data.csv'))

# Ontologies

Each document has two types of tags
1. annotation - information about annotations found in the document 
    Each annoatation has 4 tags:
    a. mention ( from this we get the id of the mention)
    b. span - we get the beginning and the end of the annoation
    c. annotator - info about annoatator (not used)
    d. spannedText - not used
2. classMention - information about the class tagged 
    id - the id of the annoatation coresponding to 1.mention
    mentionClass - has info about class id from the ontology and text of the annoatation
    

## IMPORTATNT - the text data need to be in variable texts

In [6]:
def get_folder_for_ontology(ontology):
    folder = os.path.join('concept-annotation',ontology,ontology,'knowtator')
    return folder

In [7]:
def get_data_from_file(root):
    annotations = pd.DataFrame(columns=['StartIndex', 'EndIndex','MentionTextSegment','EntityID'])
    for child in root:
    # annotation
        if child.tag=='annotation':
            tmp = {}
            id_name = None
            for c in child:
                if c.tag == 'mention':
                    id_name = c.attrib['id']
                elif c.tag == 'span':
                    tmp['StartIndex'] = c.attrib['start']
                    tmp['EndIndex'] = c.attrib['end']
            annotations.loc[id_name,['StartIndex','EndIndex']] = tmp

    # classmention
        else:
            id_name =child.attrib['id']
            tmp = {}
            for c in child:
                if 'id' in c.attrib.keys():
                    tmp['MentionTextSegment'] = c.text
                    tmp['EntityID'] = c.attrib['id']
                    annotations.loc[id_name,['MentionTextSegment','EntityID']] = tmp
    return annotations

In [13]:
def get_before_max(data,max_line):
    # get records where StartIndex not empty and smaller than max_line
    data = data[~pd.isna(data['StartIndex'])]
    return data[data['StartIndex'].astype(int) < max_line]

In [32]:
def get_data(ontology,file_name):
    folder = get_folder_for_ontology(ontology)
    data = pd.DataFrame()
    for file in os.listdir(folder):
        tree = ET.parse(os.path.join(folder,file))
        root = tree.getroot()
        annotations = get_data_from_file(root)
        annotations['PMID'] = file[:8]
        data = pd.concat([data,annotations])
    file_path = os.path.join('data',ontology,file_name)
    isExist = os.path.exists(os.path.join('data',ontology))
    if not isExist:
        os.makedirs(os.path.join('data',ontology))
    max_lines = list(texts[texts.PMID == file[:8]].MaxLine)[1]
    data = get_before_max(data,max_lines)
    data.to_csv(file_path)
    return data

In [34]:
d = get_data('CL','annotations.csv')